# My Recommender

## Data Retrieval & Preprocessing

In [9]:
# Importing Libraries

import zipfile
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [10]:
# Fetching data files
!wget 'http://eigentaste.berkeley.edu/dataset/jester_dataset_1_1.zip'


# Unzipping Data
with zipfile.ZipFile(f'jester_dataset_1_1.zip',"r") as zip_ref:
  zip_ref.extractall()

--2021-04-14 22:08:22--  http://eigentaste.berkeley.edu/dataset/jester_dataset_1_1.zip
Resolving eigentaste.berkeley.edu (eigentaste.berkeley.edu)... 128.32.192.72
Connecting to eigentaste.berkeley.edu (eigentaste.berkeley.edu)|128.32.192.72|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4171628 (4.0M) [application/zip]
Saving to: ‘jester_dataset_1_1.zip.2’

jester_dataset_1_1. 100%[===================>]   3.98M  7.04MB/s    in 0.6s    

2021-04-14 22:08:23 (7.04 MB/s) - ‘jester_dataset_1_1.zip.2’ saved [4171628/4171628]



In [11]:
# Creating DataFrame

data = pd.read_excel('/content/jester-data-1.xls', header=None, na_values=99)
data.drop(columns=0, inplace=True)
data.head(5)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.42,NaN,NaN,NaN,-7.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,NaN,8.25,NaN,NaN,7.48,7.28,7.28,8.93,NaN,6.17,7.28,NaN,NaN,8.98,7.33,NaN,6.17,9.08,7.33,...,6.46,7.28,NaN,NaN,7.04,7.28,NaN,7.28,8.25,NaN,NaN,NaN,NaN,NaN,NaN,8.93,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.03,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,NaN,NaN,NaN,NaN,2.91,3.93,6.75,6.60,NaN,6.65,-6.12,NaN,7.57,6.21,6.65,NaN,-8.30,7.18,2.82,...,0.00,-3.69,NaN,NaN,7.82,0.24,NaN,7.28,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,-2.33,NaN,NaN,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,NaN,NaN,NaN,NaN,NaN,NaN,4.13,NaN,NaN,NaN,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [12]:
# Checking Shape of data

data.shape

(24983, 100)

In [13]:
# Finding Average Rating for Jokes

score = []
for i in data.columns:
  score.append(np.nanmean(data.loc[:,i]))


#### Best & Worst Jokes

In [14]:
# Finding Best and Worst Jokes

rdict = {'JokeID':data.columns, 'Avg_Rating':score}
rating = pd.DataFrame.from_dict(rdict)
rating.sort_values(by='Avg_Rating', ascending=False, inplace=True)
print(f'Best Joke:\n {rating.head(1)}')
print(f'Worst Joke:\n {rating.tail(1)}')

Best Joke:
     JokeID  Avg_Rating
49      50    3.665085
Worst Joke:
     JokeID  Avg_Rating
57      58    -3.83388


In [15]:
# Rating for Joke 10 by 18109

data.iloc[18108,10]

-1.8

In [16]:
# Creating np.array object of data

arr = data.values
arr

array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

#### Helper Function Setup and Data Preparation

In [17]:
# Helper Function Setup

def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with np.NaN's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=np.NaN)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [18]:
new_arr, idx = replace(arr, 0.1)

## Latent Factor Models for Collaborative Filtering

In [19]:
# Creating latent factors

n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [20]:
latent_item_features

array([[0.87335534, 0.53201851],
       [0.55940786, 0.76002072],
       [0.62557164, 0.39343012],
       [0.98638855, 0.85022742],
       [0.7483003 , 0.70565987],
       [0.77473556, 0.74587727],
       [0.43008604, 0.00480812],
       [0.02115265, 0.47275401],
       [0.86977926, 0.85040446],
       [0.96823561, 0.19772564],
       [0.69043945, 0.28744729],
       [0.75768005, 0.01040225],
       [0.58156191, 0.77779273],
       [0.10907282, 0.34517074],
       [0.13419161, 0.51232672],
       [0.19142955, 0.02269376],
       [0.58132627, 0.77747978],
       [0.86184393, 0.11788089],
       [0.00260367, 0.9006093 ],
       [0.73585982, 0.17465839],
       [0.99976679, 0.44908984],
       [0.11995089, 0.86436824],
       [0.68311628, 0.48246929],
       [0.21425079, 0.4082727 ],
       [0.63162296, 0.31167243],
       [0.1625951 , 0.3967096 ],
       [0.09186189, 0.69384991],
       [0.97127078, 0.56354411],
       [0.94956689, 0.68433431],
       [0.08445957, 0.10247659],
       [0.

In [21]:
latent_user_preferences

array([[0.99143619, 0.69203986],
       [0.54091975, 0.45479908],
       [0.77712679, 0.30039557],
       ...,
       [0.8982052 , 0.08370299],
       [0.76835662, 0.6930978 ],
       [0.90321455, 0.51714204]])

In [22]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating) and rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [ ]:
# Calculating MSE

num_iter = 100
hist = sgd(num_iter)
plt.figure()
plt.plot(np.arange(0, num_iter, 10), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

In [ ]:
# Making Prediction for test set

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

In [ ]:
# Creating DataFrame for Comparison

values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns

In [ ]:
# Data Comparison 

comparison_data

In [ ]:
# Data Comparison CSV

comparison_data.to_csv('rating_comparison.csv')